<a href="https://colab.research.google.com/github/cemvardar/pragmatic_google_colab/blob/main/dslab_colab_utility_chatgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install OpenAI
import openai
from google.colab import userdata
from time import sleep
import json
import time
from datetime import datetime
import concurrent.futures

In [ ]:
def check_run(client, thread_id, run_id):
    while True:
        # Refresh the run object to get the latest status
        run = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run_id
        )

        if run.status == "completed":
            # print(f"Run is completed.")
            break
        elif run.status == "expired":
            print(f"Run is expired.")
            break
        elif run.status == "failed":
            print(f"Run is failed.")
            break
        else:
            # print(f"OpenAI: Run is not yet completed. Waiting...{run.status}")
            time.sleep(1)  # Wait for 1 second before checking again

def get_file_name_for_export_with_date_time(file_name_header, file_extenstion):
    formatted_datetime = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
    file_name = f"{file_name_header}_{formatted_datetime}.{file_extenstion}"
    file_name = file_name.replace(' ', '_')
    return file_name


def last_chatgpt_reponse(messages):
    return messages.data[0].content[0].text.value

class ChatGpt_Assitant():
    def __init__(self, name, instructions, tools, model):
        self.client = openai.Client(api_key=userdata.get('openai_api_key'))
        self.assistant = self.client.beta.assistants.create(
            name=name,
            instructions=instructions,
            tools=tools,
            model=model
        )
        self.thread = self.client.beta.threads.create()
        self.messages = None

    def add_user_input(self, user_input, print_repsonse=True):
        message = self.client.beta.threads.messages.create(thread_id=self.thread.id,
                                                    role="user",
                                                    content=user_input
        )

        run = self.client.beta.threads.runs.create(
            thread_id=self.thread.id,
            assistant_id=self.assistant.id,
        )

        check_run(self.client, self.thread.id, run.id)

        self.messages = self.client.beta.threads.messages.list(
            thread_id=self.thread.id
        )


        assistant_message = self.messages.data[0].content[0].text.value
        if print_repsonse:
            print(len(self.messages.data))
            print(assistant_message)

        return last_chatgpt_reponse(self.messages)


def get_chatgpt_shopify_helper():
    name = "Shopify helper"
    instructions="""You are a shopify site content generator for products and product collections.
                    You generate  product titles and descriptions using the details provived by the users.
                    you follow instructions and feedback from the users to generate the best titles and descriptions
                    for maximizing product sales.
                    """
    tools=[{"type": "code_interpreter"}]
    model="gpt-4o"
    return ChatGpt_Assitant(name, instructions, tools, model)


def parse_json_from_gpt_response(message):
    start_index = message.find('```json') + 8
    end_index = message.rfind("```")
    json_content = json.loads(message[start_index:end_index].replace('\n\n', ''))
    return json_content


In [ ]:
def get_json_response_from_chat_gpt(prompt):
    gpt_assistant = get_chatgpt_shopify_helper()
    message = gpt_assistant.add_user_input(prompt, False)
    json_response = parse_json_from_gpt_response(message)
    return json_response

def get_image_description_json(image_url, prompt=None):
    if prompt is None:
        prompt = """Describe what's in the photo. Respond with a JSON object. key will be image_description"""
    message_content= [{ "type": "text",
                         "text": prompt},
                      {"type": "image_url",
                       "image_url": {"url":image_url,
                       "detail": "high"}}]
    return get_json_response_from_chat_gpt(message_content)

In [ ]:
def get_json_response_for_prompt_parallel(prompt_key, prompt, max_retries=3):
    for attempt in range(max_retries):
        try:
            return prompt_key, get_json_response_from_chat_gpt(prompt)
        except Exception as e:
            print(f"Attempt {attempt+1} failed for {prompt_key}: {e}")
    return prompt_key, None  # Return None if all retries fail

def get_json_response_for_image_parallel(prompt_key, image_url, prompt, max_retries=3):
    for attempt in range(max_retries):
        try:
            return prompt_key, get_image_description_json(image_url, prompt)
        except Exception as e:
            print(f"Attempt {attempt+1} failed for {prompt_key}: {e}")
    return prompt_key, None  # Return None if all retries fail

def get_gpt_json_image_reponses_parallel(prompts_key_image_url_prompt, max_retries=3):
    responses = {}
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(get_json_response_for_image_parallel, key, image_url, prompt, max_retries)
                      for key, image_url, prompt in prompts_key_image_url_prompt]
        for future in concurrent.futures.as_completed(futures):
            key, result = future.result()
            responses[key] = result

    return responses

def get_gpt_json_reponses_parallel(prompts, max_retries=3):
    responses = {}
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(get_json_response_for_prompt_parallel, key, prompt, max_retries) for key, prompt in prompts]
        for future in concurrent.futures.as_completed(futures):
            key, result = future.result()
            responses[key] = result

    return responses


In [ ]:
# image_url = "https://storage.googleapis.com/decision-science-lab-bucket/liplips/roma_silvers_products/rs2502/2581.jpg"
# prompt = """
# we'll be creating a new photo of the jewelery in the photo changing the bacground using an AI system.
# Expand the following prompt to include different details about shows and lighting and appearaence to generate
# a minimalistic and professional and cute looking photo.
# ```prompt
# on a white wood table with 1 dry flower
# ```
# Respond with a JSON object. key will be image_prompt
# """
# # get_json_response_for_image_parallel(0, image_url, prompt)
# image_prompts = [(0, image_url, prompt), (1, image_url, prompt), (2, image_url, None)]
# get_gpt_json_image_reponses_parallel(image_prompts)

Run is completed.
Run is completed.
Run is completed.


{0: {'image_prompt': "A delicate gold and blue gemstone necklace with pearl accents displayed on a white wood table with a single dry flower. The lighting is natural and soft, creating subtle shadows that enhance the intricate details of the necklace. The background is minimalistic and professional, focusing the viewer's attention on the elegant craftsmanship of the jewelry. The dry flower adds a cute and charming touch to the composition, making it ideal for a high-end product showcase."},
 1: {'image_prompt': 'A delicate jewelry necklace placed on a white wood table with a single dry flower beside it. The scene is set with soft natural lighting from a window at an angle, casting gentle shadows that enhance the texture of the pearls, gold, and blue beads without overpowering the composition. The minimalistic backdrop and the clean white wood surface, complemented by the dry flower, create a professional yet cute and elegant aesthetic.'},
 2: {'image_description': 'The image shows a de

In [ ]:
# prompt_title= """
# Find a shopify product title and description including details about stone properties and
# possible styles for the following product. title and description will be in english.
# İnanna Lapis Kolye - Gold
# {'*Ürünlerimiz el yapımıdır.', '*İnci ve Lapis taşı kullanılmıştır.', '925 Gümüş/Silver', '*Ürünlerimiz 925 ayar gümüştür.', '*Sarı Altın kaplama yapılmıştır.', "*Kolye zincir uzunluğu 41 cm'dir. 5 cm uzatma zinciri vardır."}
# '
# Respond with a JSON object. keys will be title and description.
# don't use column in text
# """
# prompts = [('a', prompt_title), ('b', prompt_title), ('c', prompt_title)]
# get_gpt_json_reponses_parallel(prompts)

Run is completed.
Run is completed.
Run is completed.


{'a': {'title': 'Inanna Lapis Necklace - Gold',
  'description': 'Discover the elegance of the Inanna Lapis Necklace - Gold, a blend of handcrafted artistry and natural beauty. This exquisite necklace features authentic Lapis Lazuli and Pearls, known for their calming and balance-enhancing properties. Made from 925 sterling silver, the necklace is beautifully adorned with luxurious gold plating. The chain measures 41 cm long and includes a 5 cm extension chain for perfect adjustability. Every piece is handmade, ensuring that you receive a one-of-a-kind accessory that complements any style, be it casual or formal.'},
 'c': {'title': 'Inanna Lapis Necklace - Gold',
  'description': 'Handcrafted with care, our Inanna Lapis Necklace in Gold features a luxurious blend of pearl and lapis stone. Made from 925 sterling silver and plated with yellow gold, this necklace embodies timeless elegance and sophistication. The chain is 41 cm long with a 5 cm extension, offering versatility in styling. 

In [ ]:
# gpt_assistant = get_chatgpt_shopify_helper()
# gpt_assistant.add_user_input('Respond with a JSON object. Content for an example product', False)
# message

OpenAI: Run is not yet completed. Waiting...queued
Run is completed.


In [ ]:
# prompt_title= """
# Find a shopify product title and description including details about stone properties and
# possible styles for the following product. title and description will be in english.
# İnanna Lapis Kolye - Gold
# {'*Ürünlerimiz el yapımıdır.', '*İnci ve Lapis taşı kullanılmıştır.', '925 Gümüş/Silver', '*Ürünlerimiz 925 ayar gümüştür.', '*Sarı Altın kaplama yapılmıştır.', "*Kolye zincir uzunluğu 41 cm'dir. 5 cm uzatma zinciri vardır."}
# '
# Respond with a JSON object. keys will be title and description.
# don't use column in text
# """

In [ ]:
# gpt_assistant = get_chatgpt_shopify_helper()
# message = gpt_assistant.add_user_input(prompt_title, False)
# print(message)
# parse_json_from_gpt_response(message)

OpenAI: Run is not yet completed. Waiting...queued
Run is completed.
```json
{
  "title": "Inanna Lapis Necklace - Gold",
  "description": "Our products are handmade. Featuring pearl and lapis stones. Made with 925 sterling silver and coated with yellow gold. The chain length is 41 cm with an additional 5 cm extender. The lapis stone is known for its deep blue color and is believed to provide wisdom and truth. The pearls symbolize purity and elegance. This versatile necklace can be styled with both casual and formal outfits, adding a touch of sophistication and grace."
}
```


{'title': 'Inanna Lapis Necklace - Gold',
 'description': 'Our products are handmade. Featuring pearl and lapis stones. Made with 925 sterling silver and coated with yellow gold. The chain length is 41 cm with an additional 5 cm extender. The lapis stone is known for its deep blue color and is believed to provide wisdom and truth. The pearls symbolize purity and elegance. This versatile necklace can be styled with both casual and formal outfits, adding a touch of sophistication and grace.'}

In [ ]:
# bp_prompt = '''
# give 6 bullet point items in english for the following product
# İnanna Lapis Kolye - Gold
# {'*Ürünlerimiz el yapımıdır.', '*İnci ve Lapis taşı kullanılmıştır.', '925 Gümüş/Silver', '*Ürünlerimiz 925 ayar gümüştür.', '*Sarı Altın kaplama yapılmıştır.', "*Kolye zincir uzunluğu 41 cm'dir. 5 cm uzatma zinciri vardır."}
# Respond with a JSON object. key will be items
# don't use column in text
# '''
# message = gpt_assistant.add_user_input(bp_prompt, False)
# print(message)
# parse_json_from_gpt_response(message)

OpenAI: Run is not yet completed. Waiting...queued
Run is completed.
```json
{
  "items": [
    "Handmade craftsmanship",
    "Incorporates pearl and lapis stones",
    "Constructed with 925 sterling silver",
    "Yellow gold plating",
    "Necklace chain length is 41 cm with 5 cm extender",
    "Lapis stone symbolizes wisdom and truth; pearls symbolize purity and elegance"
  ]
}
```


{'items': ['Handmade craftsmanship',
  'Incorporates pearl and lapis stones',
  'Constructed with 925 sterling silver',
  'Yellow gold plating',
  'Necklace chain length is 41 cm with 5 cm extender',
  'Lapis stone symbolizes wisdom and truth; pearls symbolize purity and elegance']}

In [ ]:
# image_url = "https://storage.googleapis.com/decision-science-lab-bucket/liplips/roma_silvers_products/rs2502/2581.jpg"
# get_image_description_json(image_url)

OpenAI: Run is not yet completed. Waiting...in_progress
Run is completed.


{'image_description': 'The photo features a delicate necklace. The necklace is composed of small blue and white beads. Along the length of the necklace, there are various charms, including white pearl-like beads and small golden pendants shaped like the Eiffel Tower. Additionally, there is a prominent decorative element, a textured gold fan-shaped pendant, towards the bottom center of the necklace.'}

In [ ]:
# image_url = "https://storage.googleapis.com/decision-science-lab-bucket/liplips/roma_silvers_products/rs2502/2581.jpg"
# prompt = """
# we'll be creating a new photo of the jewelery in the photo changing the bacground using an AI system.
# Expand the following prompt to include different details about shows and lighting and appearaence to generate
# a minimalistic and professional and cute looking photo.
# ```prompt
# on a white wood table with 1 dry flower
# ```
# Respond with a JSON object. key will be image_prompt
# """
# get_image_description_json(image_url, prompt)

OpenAI: Run is not yet completed. Waiting...in_progress
Run is completed.


{'image_prompt': "A delicate gold and pearl necklace laid out elegantly on a white wood table with 1 dry flower beside it. The scene is bathed in soft natural light, creating gentle shadows and a warm, inviting atmosphere. The table's minimalistic texture contrasts beautifully with the intricate details of the necklace. The dry flower adds a touch of rustic charm, enhancing the overall minimalistic and professional aesthetic. The focus is sharp on the necklace, with a slight bokeh effect on the dry flower and background, making the jewelry stand out prominently while maintaining a cute and refined look."}